## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [39]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [123]:
df = pd.read_csv(r"../Day073-erf/002 QueryResults.csv", names=['DATE', 'TAG', 'POSTS'])
df

,DATE,TAG,POSTS
0,m,TagName,NaN
1,2008-07-01 00:00:00,c#,3.00
2,2008-08-01 00:00:00,assembly,8.00
3,2008-08-01 00:00:00,javascript,162.00
4,2008-08-01 00:00:00,c,85.00
...,...,...,...
1987,2020-07-01 00:00:00,r,"5,694.00"
1988,2020-07-01 00:00:00,go,743.00
1989,2020-07-01 00:00:00,ruby,775.00
1990,2020-07-01 00:00:00,perl,182.00


## **Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [5]:
df.head()

,DATE,TAG,POSTS
0,m,TagName,NaN
1,2008-07-01 00:00:00,c#,3.0
2,2008-08-01 00:00:00,assembly,8.0
3,2008-08-01 00:00:00,javascript,162.0
4,2008-08-01 00:00:00,c,85.0


In [6]:
df.tail()

,DATE,TAG,POSTS
1987,2020-07-01 00:00:00,r,5694.0
1988,2020-07-01 00:00:00,go,743.0
1989,2020-07-01 00:00:00,ruby,775.0
1990,2020-07-01 00:00:00,perl,182.0
1991,2020-07-01 00:00:00,swift,3607.0


## **Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [7]:
df.shape

(1992, 3)

## **Challenge**: Count the number of entries in each column of the dataframe

In [9]:
len(df)

1992

In [17]:
df.groupby('TAG').value_counts()

TAG       DATE                 POSTS 
assembly  2008-08-01 00:00:00  8.0       1
          2008-09-01 00:00:00  28.0      1
          2008-10-01 00:00:00  15.0      1
          2008-11-01 00:00:00  17.0      1
          2008-12-01 00:00:00  18.0      1
                                        ..
swift     2020-03-01 00:00:00  3434.0    1
          2020-04-01 00:00:00  4015.0    1
          2020-05-01 00:00:00  4066.0    1
          2020-06-01 00:00:00  3733.0    1
          2020-07-01 00:00:00  3607.0    1
Name: count, Length: 1991, dtype: int64

## **Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [62]:
all_languages = df['TAG'].drop_duplicates()[1:].to_list()
meta_df = pd.DataFrame(
    data= {lang: df[df['TAG'] == lang]['POSTS'].sum() for lang in all_languages}.values(),
    index= all_languages,
    columns=['ALL TIME POSTS NUMBER']
).sort_values('ALL TIME POSTS NUMBER', ascending=False)
meta_df


,ALL TIME POSTS NUMBER
javascript,"2,056,510.00"
java,"1,696,403.00"
python,"1,496,210.00"
c#,"1,423,530.00"
php,"1,361,988.00"
c++,"684,210.00"
r,"356,799.00"
c,"336,042.00"
swift,"273,055.00"
ruby,"214,582.00"


In [90]:
meta_df[meta_df['ALL TIME POSTS NUMBER']==meta_df['ALL TIME POSTS NUMBER'].max()]

,ALL TIME POSTS NUMBER
javascript,"2,056,510.00"


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

## **Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [81]:
month_df = pd.DataFrame(
    data= {lang: df[df['TAG'] == lang]['DATE'].drop_duplicates().count() for lang in all_languages}.values(),
    index= all_languages,
    columns=['HOW MANY MONTHS']
).sort_values('HOW MANY MONTHS')
month_df

,HOW MANY MONTHS
go,129
swift,135
r,142
assembly,144
c++,144
javascript,144
c,144
python,144
php,144
java,144


In [84]:
month_df[month_df['HOW MANY MONTHS'] == month_df['HOW MANY MONTHS'].min()]

,HOW MANY MONTHS
go,129


## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [139]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date
df

,DATE,TAG,POSTS
1,2008-07-01,c#,3.00
2,2008-08-01,assembly,8.00
3,2008-08-01,javascript,162.00
4,2008-08-01,c,85.00
5,2008-08-01,python,124.00
...,...,...,...
1987,2020-07-01,r,"5,694.00"
1988,2020-07-01,go,743.00
1989,2020-07-01,ruby,775.00
1990,2020-07-01,perl,182.00


## Data Manipulation



**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 